In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
from utils.utils import save_experiment, train_and_evaluate_logistic_regression, evaluate_model_on_parquet
from configs.config_local import DATASET_PATH, ITW_DATASET_PATH

In [2]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

lr_params = {
            "C": 10.0,  # Regularization strength
            "class_weight": "balanced",  # Handle imbalanced classes
            "max_iter": 1000,  # Usually enough to converge
            "random_state": 42,
            "solver": "liblinear",  # Good for small-medium datasets, handles binary classification well
            "penalty": "l2",  # Standard L2 regularization
        }

pipeline, metrics, lr_params, feature_names, metadata_extra = train_and_evaluate_logistic_regression(train_data_path, test_data_path, lr_params)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.984039   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.599396   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -309.998932   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -329.094482   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -369.833069   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0  106.660751   134.430344   62.437195     9.798609   39.744644    13.036348   
1  115.704865   106.857346   50.405762    12.835211   42.213852    37.456020   
2   72.639381   143.500931   32.701843    16.544865   31.033148    49.439816   
3  119.318130    90.687096   70.433701    12.409759   38.385540    40.940025   
4  129.029114    77.314659   67.398788     2.168967   48.882267    34.795979   

   mfcc_std_3  ...  mel_spectrogram_mean_123  mel_spectrogram_std_12

In [3]:
print(metrics)

{'accuracy': 0.8109624514458351, 'precision': 0.872382851445663, 'recall': 0.7383966244725738, 'f1': 0.810374659369997, 'roc_auc': 0.8960575360438938}


In [4]:
save_experiment(
    model=pipeline,
    metrics=metrics,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments",  "logistic_reg",),
    model_params=lr_params,
    feature_names=feature_names,
    metadata_extra=metadata_extra,
)


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\logistic_reg\exp_20260111_195634


'c:\\Users\\konst\\Documents\\GitHub\\audio-deepfake-detection\\notebooks\\experiments\\logistic_reg\\exp_20260111_195634'

In [5]:
# test model on itw data
test_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
metrics,_=evaluate_model_on_parquet(pipeline, test_path)
metrics

{'accuracy': 0.6658306612908347,
 'precision': 0.5711204449994324,
 'recall': 0.4271160539943968,
 'f1': 0.6202675598200457,
 'roc_auc': 0.6838180253233701}

#### Train model without mel spectrogram featuress

In [6]:
train_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128.parquet")
test_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128.parquet")

train_data = pd.read_parquet(train_data_path)
test_data = pd.read_parquet(test_data_path)
print(train_data.columns)
print('---------------------------------------')
print(test_data.columns)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'mel_spectrogram_mean_123', 'mel_spectrogram_std_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_std_124',
       'mel_spectrogram_mean_125', 'mel_spectrogram_std_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_std_126',
       'mel_spectrogram_mean_127', 'mel_spectrogram_std_127'],
      dtype='object', length=512)
---------------------------------------
Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'mel_spectrogram_mean_123', 'mel_spectrogram_std_123',
       'mel_spectrogram_mean_124', 'mel_spectrogram_std_124',
       'mel_spectrogram_mean_125', 'mel_spectrogram_std_125',
       'mel_spectrogram_mean_126', 'mel_spectrogram_std_126',
       'mel_spectrogram_mean_127', 'mel_spe

In [7]:
train_data_no_mel = train_data.loc[:, ~train_data.columns.str.startswith("mel_spectrogram")]
print(train_data_no_mel.columns)
print('---------------------------------------')
test_data_no_mel = test_data.loc[:, ~test_data.columns.str.startswith("mel_spectrogram")]
print(test_data_no_mel.columns)

Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'spectral_centroid_mean_0', 'spectral_centroid_std_0',
       'spectral_bandwidth_mean_0', 'spectral_bandwidth_std_0',
       'spectral_flatness_mean_0', 'spectral_flatness_std_0',
       'spectral_rolloff_mean_0', 'spectral_rolloff_std_0', 'pitch_yin_mean_0',
       'pitch_yin_std_0'],
      dtype='object', length=256)
---------------------------------------
Index(['label', 'filename', 'mfcc_mean_0', 'mfcc_std_0', 'mfcc_mean_1',
       'mfcc_std_1', 'mfcc_mean_2', 'mfcc_std_2', 'mfcc_mean_3', 'mfcc_std_3',
       ...
       'spectral_centroid_mean_0', 'spectral_centroid_std_0',
       'spectral_bandwidth_mean_0', 'spectral_bandwidth_std_0',
       'spectral_flatness_mean_0', 'spectral_flatness_std_0',
       'spectral_rolloff_mean_0', 'spectral_rolloff_std_0', 'pitch_yin_mean_0',
       'pitch_yin_std_0'],
      dtype=

In [8]:
save_train_no_mel_data_path = os.path.join(DATASET_PATH, "training_features_40_512_256_128_no_mel.parquet")
train_data_no_mel.to_parquet(save_train_no_mel_data_path)

save_test_no_mel_data_path = os.path.join(DATASET_PATH, "testing_features_40_512_256_128_no_mel.parquet")
test_data_no_mel.to_parquet(save_test_no_mel_data_path)


In [9]:
train_data_path = save_train_no_mel_data_path
test_data_path = save_test_no_mel_data_path

df = pd.read_parquet(train_data_path)
df_test = pd.read_parquet(test_data_path)
#print(df.head())
print(df_test.head())
print(df_test.shape)
#df.shape()

lr_params_no_mel = {
            "C": 1.0,  # Regularization strength
            "class_weight": "balanced",  # Handle imbalanced classes
            "max_iter": 1000,  # Usually enough to converge
            "random_state": 42,
            "solver": "liblinear",  # Good for small-medium datasets, handles binary classification well
            "penalty": "l2",  # Standard L2 regularization
        }

pipeline_no_mel, metrics_no_mel, lr_params_no_mel, feature_names_no_mel, metadata_extra_no_mel = train_and_evaluate_logistic_regression(train_data_path, test_data_path, lr_params)

  label                                           filename  mfcc_mean_0  \
0  real    file1.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.984039   
1  real   file10.wav_16k.wav_norm.wav_mono.wav_silence.wav  -335.599396   
2  real  file100.wav_16k.wav_norm.wav_mono.wav_silence.wav  -309.998932   
3  real  file1000.wav_16k.wav_norm.wav_mono.wav_silence...  -329.094482   
4  real  file1001.wav_16k.wav_norm.wav_mono.wav_silence...  -369.833069   

   mfcc_std_0  mfcc_mean_1  mfcc_std_1  mfcc_mean_2  mfcc_std_2  mfcc_mean_3  \
0  106.660751   134.430344   62.437195     9.798609   39.744644    13.036348   
1  115.704865   106.857346   50.405762    12.835211   42.213852    37.456020   
2   72.639381   143.500931   32.701843    16.544865   31.033148    49.439816   
3  119.318130    90.687096   70.433701    12.409759   38.385540    40.940025   
4  129.029114    77.314659   67.398788     2.168967   48.882267    34.795979   

   mfcc_std_3  ...  spectral_centroid_mean_0  spectral_centroid_std_

In [10]:
print(metrics_no_mel)

{'accuracy': 0.6793267155804921, 'precision': 0.7525714285714286, 'recall': 0.5556962025316455, 'f1': 0.6753322998468629, 'roc_auc': 0.7394593043193034}


In [11]:
save_experiment(
    model=pipeline_no_mel,
    metrics=metrics_no_mel,
    experiment_dir=os.path.join(sys.path[0], "notebooks", "experiments", "logistic_reg"),
    model_params=lr_params_no_mel,
    feature_names=feature_names_no_mel,
    metadata_extra=metadata_extra_no_mel,
)


Experiment saved to: c:\Users\konst\Documents\GitHub\audio-deepfake-detection\notebooks\experiments\logistic_reg\exp_20260111_195645


'c:\\Users\\konst\\Documents\\GitHub\\audio-deepfake-detection\\notebooks\\experiments\\logistic_reg\\exp_20260111_195645'

In [12]:
# test model on itw data

# remove mel features and save new df
test_data_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_trimmed.parquet")
test_data = pd.read_parquet(test_data_path)
test_data_no_mel = test_data.loc[:, ~test_data.columns.str.startswith("mel_spectrogram")]
save_test_data_no_mel_path = os.path.join(ITW_DATASET_PATH, 'normalized_features',"itw_features_40_2048_512_128_no_mel_trimmed.parquet")
test_data_no_mel.to_parquet(save_test_data_no_mel_path)

#evaluate the model
test_path = save_test_data_no_mel_path
metrics,_ = evaluate_model_on_parquet(pipeline_no_mel, test_path)
metrics

{'accuracy': 0.6264960792406108,
 'precision': 0.5005586592178771,
 'recall': 0.5324730452500213,
 'f1': 0.6059656349615231,
 'roc_auc': 0.6302691863007136}